##### 인스타 해시태그 크롤링 ##

## 1. 인스타그램 특정 계정의 최근 7일간의 게시글 정보 크롤링

In [12]:
pip install selenium pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
from datetime import datetime, timedelta

# 1. 웹드라이버 설정 및 인스타그램 로그인
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 인스타그램 로그인 페이지로 이동
driver.get("https://www.instagram.com/accounts/login/")
time.sleep(5)  # 페이지 로드를 기다림

# 로그인 정보 입력 (수동 로그인을 위해 시간을 더 줌)
username = "﻿snsdkf1234@naver.com"
password = "wel1234"

driver.find_element(By.NAME, "﻿snsdkf1234@naver.com").send_keys(username)
driver.find_element(By.NAME, "wel1234").send_keys(password)
driver.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button').click()
time.sleep(10)  # 로그인 후 대기

# 2. 특정 계정으로 이동
account_name = "seoul___nadri"  # 크롤링할 계정의 사용자 이름
driver.get(f"https://www.instagram.com/{account_name}/")
time.sleep(5)

# 3. 최근 7일간의 게시물 탐색 및 데이터 추출
posts_data = []
now = datetime.now()

# 게시물 URL 리스트 가져오기
posts = driver.find_elements(By.XPATH, '//a[contains(@href, "/p/")]')

for post in posts:
    post_url = post.get_attribute('href')
    driver.get(post_url)
    time.sleep(5)  # 게시물 페이지 로드 대기

    # 4. 게시물 데이터 추출
    try:
        likes = driver.find_element(By.XPATH, '//div[contains(@class, "Nm9Fw")]/button/span').text
    except:
        likes = '0'
    
    hashtags = [tag.text for tag in driver.find_elements(By.XPATH, '//a[contains(@href, "/explore/tags/")]')]
    hashtags = hashtags[:10]  # 상위 10개 해시태그만 가져오기
    
    try:
        post_text = driver.find_element(By.XPATH, '//div[@class="C4VMK"]/span').text
    except:
        post_text = ""

    post_date_element = driver.find_element(By.XPATH, '//time')
    post_date = post_date_element.get_attribute('datetime')
    post_date = datetime.strptime(post_date, "%Y-%m-%dT%H:%M:%S.%fZ")  # ISO 형식을 datetime으로 변환
    
    # 5. 7일 내의 게시물만 필터링
    if now - timedelta(days=7) > post_date:
        break

    posts_data.append({
        'account_name': account_name,
        'likes': likes,
        'hashtags': ', '.join(hashtags),
        'post_text': post_text,
        'post_date': post_date.strftime("%Y-%m-%d %H:%M:%S"),
        'crawl_time': now.strftime("%Y-%m-%d %H:%M:%S"),
        'post_url': post_url
    })

# 6. 데이터 저장
df = pd.DataFrame(posts_data)
df.to_csv(f'{account_name}_posts.csv', index=False)

driver.quit()


ModuleNotFoundError: No module named 'webdriver_manager'

In [3]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip show webdriver-manager

Name: webdriver-manager
Version: 4.0.2
Summary: Library provides the way to automatically manage drivers for different browsers
Home-page: https://github.com/SergeyPirogov/webdriver_manager
Author: Sergey Pirogov
Author-email: automationremarks@gmail.com
License: 
Location: D:\apps\Lib\site-packages
Requires: packaging, python-dotenv, requests
Required-by: 
Note: you may need to restart the kernel to use updated packages.
